In [ ]:
import sqlite3
import pandas as pd
import os
import joblib
import warnings
warnings.filterwarnings("ignore")


# 1. Connect to your SQLite database
# Replace "your_database.db" with your actual .db file path
db_path = "MIG_Cement_Records.db"
conn = sqlite3.connect(db_path)

# 2. Write your SQL query
query = "SELECT * FROM sqlite_master;"

# 3. Fetch into a DataFrame
df = pd.read_sql_query(query, conn)

# 4. Display the data
print(df.head())

In [ ]:
# for table in ["Sites", "CementTypes", "Operations"]:
#     query = f"PRAGMA table_info({table});"
#     print(f"\nColumns in {table}:")
#     print(pd.read_sql_query(query, conn))

for table in ["Sites", "CementTypes", "Operations"]:
    print(f"\nPreview of {table}:")
    print(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn))

In [ ]:
query = """
SELECT 
    o.date,
    o.site_id,
    s.region,
    s.behavior,
    o.cement_type,
    o.planned_pour_tonnes,
    o.consumed_tonnes,
    o.opening_inventory_tonnes,
    o.deliveries_tonnes,
    o.closing_inventory_tonnes,
    o.rain_mm,
    o.avg_temp_c,
    o.silo_capacity
FROM Operations o
JOIN Sites s ON o.site_id = s.site_id
JOIN CementTypes c ON o.cement_type = c.cement_type;
"""

df_joined = pd.read_sql_query(query, conn)
print(df_joined)  # Should now show (32880, 11) if that’s the actual data
# conn.close()



In [ ]:
# Save DataFrame to CSV
df_joined.to_csv("cement_records_joined.csv", index=False)

print("Data saved to cement_records_joined.csv")

In [ ]:
df = pd.read_csv("cement_records_joined.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
print("shape:",df.shape)
display(df.columns.tolist())
display(df.isnull().sum())

In [ ]:
numerical_col = df.select_dtypes(include=["int64",'float64']).columns
numerical_col

In [ ]:
(df[numerical_col] < 0).sum()

In [ ]:
df['inventory_check'] = (
    df['opening_inventory_tonnes'] + df['deliveries_tonnes'] -df['consumed_tonnes']).round(2) == df['closing_inventory_tonnes'].round(2)

df['inventory_check'].mean()

In [ ]:
df["stock_out"] = df["planned_pour_tonnes"] >(df['closing_inventory_tonnes'] + df['deliveries_tonnes'])

In [ ]:
df['over_capacity'] = df['closing_inventory_tonnes'] > df['silo_capacity']

In [ ]:
# df['idle'] = ((df['planned_pour_tonnes'] == 0 & df['deliveries_tonnes'] == 0))

df['idle'] = (df['planned_pour_tonnes'] == 0) & (df['deliveries_tonnes'] == 0)


In [ ]:
df['waste_risk'] = (df['closing_inventory_tonnes'] > 0.85 * df['silo_capacity']) & (df['planned_pour_tonnes'] < 5)


In [ ]:
df['pour_disrupted'] = (df['planned_pour_tonnes'] > 0) & (df['consumed_tonnes'] == 0)

In [ ]:
kpi_summary = df.groupby('site_id').agg(
    total_days =("date",'count'),
    total_consumed_tonnes =('consumed_tonnes','sum'),
    avg_daily_consumed = ('consumed_tonnes','mean'),
    stockout_pct = ('stock_out',lambda x : round(x.mean()*100,2)),
    overcapacity = ('over_capacity',lambda x : round(x.mean()*100,2)),
    idle_pct = ('idle',lambda x : round(x.mean()*100,2)),
    waste_risk_pct = ('waste_risk',lambda x : round(x.mean()*100,2)),
    pour_disrupted_pct = ('pour_disrupted',lambda x : round(x.mean()*100,2)),
    silo_capacity = ('silo_capacity','first'),
    region = ('region','first')
).reset_index()



kpi_summary.head()

In [ ]:
# Top 5 Sites by Average Daily Consumption

top5_consumption = kpi_summary.sort_values(
    by="avg_daily_consumed", ascending=False
).head(5)

print("Top 5 Sites by Avg Daily Consumption:")
print(top5_consumption[["site_id", "avg_daily_consumed", "region"]])


import matplotlib.pyplot as plt

top5_consumption = kpi_summary.nlargest(5, "avg_daily_consumed")

plt.figure(figsize=(8,5))
plt.bar(top5_consumption["site_id"], top5_consumption["avg_daily_consumed"], color="skyblue")
plt.title("Top 5 Sites by Avg Daily Consumption")
plt.xlabel("Site ID")
plt.ylabel("Avg Daily Consumption (tonnes)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Worst 5 Sites by Stockout %

worst5_stockout = kpi_summary.sort_values(
    by="stockout_pct", ascending=False
).head(5)

print("Worst 5 Sites by Stockout %:")
print(worst5_stockout[["site_id", "stockout_pct", "region"]])




In [ ]:
# Sites with Highest Pour Disruption

top5_total1 = kpi_summary.nlargest(5, "total_consumed_tonnes")
print("Top 5 Sites by Total Consumption:")
print(top5_total1[["site_id", "total_consumed_tonnes", "region"]])


top5_total = kpi_summary.nlargest(15, "total_consumed_tonnes")

plt.figure(figsize=(8,5))
plt.bar(top5_total1["site_id"], top5_total1["total_consumed_tonnes"], color="green")
plt.title("Top 5 Sites by Highest pour Distrption")
plt.xlabel("Site ID")
plt.ylabel("Total Consumed (tonnes)")
plt.xticks(rotation=45)
plt.show()



In [ ]:

top_site =kpi_summary.sort_values('total_consumed_tonnes', ascending = False).head(20)
plt.figure(figsize=(15,6))
plt.subplot(2,3,2)
plt.bar(top_site['site_id'],top_site['total_consumed_tonnes'])
plt.title =("Top 20 sited by total cement consumption")
plt.xticks(rotation=45)
plt.ylim(15000, top_site['total_consumed_tonnes'].max() *1.2)
plt.tight_layout()
plt.show()

In [ ]:
import  seaborn as sns

risk_site =kpi_summary.sort_values('stockout_pct', ascending = False).head(10)
plt.figure(figsize=(15,6))
plt.subplot(2,3,2)
sns.barplot(x ='site_id',y='stockout_pct',data = risk_site)
plt.title =("Top 10 sited by stockout Risk %")
plt.xticks(rotation=45)
plt.ylim(30,70)
# plt.ylim(15000, top_site['total_consumed_tonnes'].max() *1.2)
plt.tight_layout()
plt.show()

In [ ]:

overcapacity_site =kpi_summary.sort_values('overcapacity', ascending = False).head(10)
plt.figure(figsize=(15,6))
plt.subplot(2,3,2)
sns.barplot(x ='site_id',y='overcapacity',data = overcapacity_site)
plt.title =("Top 10 sited by overcapacity_pct %")
plt.xticks(rotation=45)
# plt.ylim(10,60)
# plt.ylim(15000, top_site['total_consumed_tonnes'].max() *1.2)
plt.tight_layout()
plt.show()

In [ ]:
idle_site =kpi_summary.sort_values('idle_pct', ascending = False).head(10)
plt.figure(figsize=(15,6))
plt.subplot(2,3,2)
sns.barplot(x ='site_id',y='idle_pct',data = idle_site)
plt.title =("Top 10 sited by IDLE")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
waste_site =kpi_summary.sort_values('waste_risk_pct', ascending = False).head(10)
plt.figure(figsize=(15,6))
plt.subplot(2,3,2)
sns.barplot(x ='site_id',y='waste_risk_pct',data = waste_site)
plt.title=("Top 10 sited by Waste Risk %")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df['utilization+pct'] = df['closing_inventory_tonnes'] / df['silo_capacity']

In [ ]:
utilization_summary = df.groupby('site_id')['utilization+pct'].mean().sort_values(ascending=False)

In [ ]:
utilization_summary.plot(kind = 'bar' , title='average silo utilization by site',figsize=(15,5))

In [ ]:
plt.figure(figsize=(15,5))

# Create subplot slot (2 rows, 3 columns, position 2)
ax = plt.subplot(2,3,2)

# Plot on that axis
utilization_summary.plot(
    kind="bar",
    x="site_id",
    y="avg_silo_utilization",   # change to your column
    title="Average Silo Utilization by Site",
    ax=ax
)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Select top 10 sites for each KPI
top10_consumed = kpi_summary.nlargest(10, "total_consumed_tonnes")
top10_avg = kpi_summary.nlargest(10, "avg_daily_consumed")
top10_stockout = kpi_summary.nlargest(10, "stockout_pct")
top10_overcap = kpi_summary.nlargest(10, "overcapacity")
top10_idle = kpi_summary.nlargest(10, "idle_pct")
top10_waste = kpi_summary.nlargest(10, "waste_risk_pct")

# Create dashboard grid (2 rows x 3 cols)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# --- Plot 1: Total Consumption ---
axes[0,0].bar(top10_consumed["site_id"], top10_consumed["total_consumed_tonnes"], color="green")
axes[0,0].set_title("Top 10 Sites by Total Consumption")
axes[0,0].set_xlabel("Site ID")
axes[0,0].set_ylabel("Tonnes")
axes[0,0].tick_params(axis="x", rotation=45)

# --- Plot 2: Avg Daily Consumption ---
axes[0,1].bar(top10_avg["site_id"], top10_avg["avg_daily_consumed"], color="skyblue")
axes[0,1].set_title("Top 10 Sites by Avg Daily Consumption")
axes[0,1].set_xlabel("Site ID")
axes[0,1].set_ylabel("Tonnes")
axes[0,1].tick_params(axis="x", rotation=45)

# --- Plot 3: Stockout % ---
axes[0,2].bar(top10_stockout["site_id"], top10_stockout["stockout_pct"], color="salmon")
axes[0,2].set_title("Top 10 Sites by Stockout %")
axes[0,2].set_xlabel("Site ID")
axes[0,2].set_ylabel("%")
axes[0,2].tick_params(axis="x", rotation=45)

# --- Plot 4: Overcapacity % ---
axes[1,0].bar(top10_overcap["site_id"], top10_overcap["overcapacity"], color="orange")
axes[1,0].set_title("Top 10 Sites by Overcapacity %")
axes[1,0].set_xlabel("Site ID")
axes[1,0].set_ylabel("%")
axes[1,0].tick_params(axis="x", rotation=45)

# --- Plot 5: Idle % ---
axes[1,1].bar(top10_idle["site_id"], top10_idle["idle_pct"], color="purple")
axes[1,1].set_title("Top 10 Sites by Idle %")
axes[1,1].set_xlabel("Site ID")
axes[1,1].set_ylabel("%")
axes[1,1].tick_params(axis="x", rotation=45)

# --- Plot 6: Waste Risk % ---
axes[1,2].bar(top10_waste["site_id"], top10_waste["waste_risk_pct"], color="red")
axes[1,2].set_title("Top 10 Sites by Waste Risk %")
axes[1,2].set_xlabel("Site ID")
axes[1,2].set_ylabel("%")
axes[1,2].tick_params(axis="x", rotation=45)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
behavior_kpi = df.groupby('behavior').agg(
    avg_stockout=('stock_out', 'mean'),
    avg_overcapacity=('over_capacity', 'mean'),
    avg_waste_risk=('waste_risk', 'mean')
)
# behavior_kpi.plot(kind='bar',figsize(15,6),title='kpi Comparison by site behavious')

behavior_kpi.plot(
    kind='bar',
    figsize=(15,6),
    title='KPI Comparison by Site Behavior'
)

plt.xticks(rotation=90)
plt.ylabel("Average Value")
plt.tight_layout()
plt.show()




In [ ]:
#kpi comparison by site Behavior
#Comparison key performace indicator across different site Behavior to identify trends

site_usage = df.groupby('site_id')['consumed_tonnes'].sum().sort_values(ascending=False)

total_usage = site_usage.sum()
top5=site_usage.head(5)
top5_share = round(top5.sum() / total_usage * 100,2)

display(top5)
display(top5_share)

In [ ]:
# Group by site and take mean of selected columns
site_usage = df.groupby('site_id')[['closing_inventory_tonnes', 'planned_pour_tonnes']].mean()

# Sort by closing inventory (you can also sort by 'stockout_pct' if you prefer)
site_usage = site_usage.sort_values('closing_inventory_tonnes', ascending=False)

# Total usage for closing inventory
total_usage = site_usage['closing_inventory_tonnes'].sum()

# Top 5 sites
top5 = site_usage.head(10)

# Percentage share of top 5 sites
top5_share = round(top5['closing_inventory_tonnes'].sum() / total_usage * 100, 2)

display(top5)
display(top5_share)


In [ ]:
#cement Type Demand share
#Understanding which cement types most consumed to optimize inventory chain.
cement_demand =df.groupby('cement_type')["consumed_tonnes"].sum().sort_values()
cement_demand.plot(kind='pie',autopct='%1.1f%%',figsize=(6,6),title='Cement Type Demand Share')

In [ ]:
#Monthly Cement Demand Seasionality
#Identifying seasonal treds in cement comparison to better plan logistic abd inventory.
df["month"] = df['date'].dt.to_period("M")
monthly_demand = df.groupby('month')['consumed_tonnes'].sum()
monthly_demand.plot(figsize=(12,5),title='Monthly cemnet Demand Seasonality')

MODELIMG

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
site_id = 'SITE_001'
site_df = df[df['site_id'] == site_id].copy()
site_df.set_index('date',inplace=True)
site_df = site_df.sort_index()

In [ ]:
print(site_df.columns)

In [ ]:
# y = site_df['consumed_tonnes']

# X= site_df[['planned_pour_tonnes', 'rain_mm', 'avg_temp_c']]

# # Train-test split first
# # split_index = int(len(site_df) * 0.8)
# # y_train, y_test = X.iloc[:split_index], X.iloc[split_index:]
# # y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
# X_test,y_test,X_train,y_train(X,y,test_size = 0.2, random_state =42)
# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)

from sklearn.model_selection import train_test_split

X = site_df[['planned_pour_tonnes', 'rain_mm', 'avg_temp_c']]
y = site_df['consumed_tonnes']  # or your target variable

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



In [ ]:
from statsmodels.tsa.ar_model import AutoReg

# Train-test split first
split_index = int(len(site_df) * 0.8)
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Fit AutoReg (only y, no exog allowed here)
model = AutoReg(y_train, lags=12)
results = model.fit()

print(results.summary())

# Forecast same length as y_test
# ar_forecast = results.predict(
#     start=y_test.index[0], 
#     end=y_test.index[-1]
# )

ar_forecast = results.predict(
    start=y_test.index[0], 
    end=y_test.index[-1]
)

# Align forecast index with y_test
ar_forecast = pd.Series(ar_forecast, index=y_test.index)


In [ ]:
ar_forecast

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    mode='lines',
    name="Actual",
    line=dict(color='black')
))

# Forecasted values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=ar_forecast,
    mode='lines',
    name="Forecast",
    line=dict(color='red', dash='dash')
))

# Layout customization
fig.update_layout(
    title=f'AUTOREG FORECAST VS ACTUAL - {site_id}',
    xaxis_title="Date",
    yaxis_title="Cement Consumed (Tonnes)",
    legend=dict(x=0, y=1),
    template='plotly_white',
    width=1000,
    height=450
)

fig.show()


In [ ]:
# Train-test split
split_index = int(len(site_df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

In [ ]:
#visualisation SARIMAX Forecast Vs actual
fig = go.Figure()

# Actual Values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    mode='lines',
    name="Actual",
    line=dict(color='black')
))

# Forecast Values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=ar_forecast,
    mode='lines',
    name='Forecast',
    line=dict(color='green', dash='dash')
))

# Layout
fig.update_layout(
    title=f'LIGHTGB FORECAST VS ACTUAL - {site_id}',
    xaxis_title="Date",
    yaxis_title="Cement Consumed (Tonnes)",
    legend=dict(x=0, y=1),
    template='plotly_white',
    width=1000,
    height=450
)

fig.show()

In [ ]:
# Evaluation Metrics
import numpy as np
from sklearn.metrics import mean_squared_error

def print_metrics(y_true, y_pred, label):
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{label} - MAPE: {mape:.2f}%, RMSE: {rmse:.2f} tonnes")


# Comparing SARIMAX and Random Forest
print_metrics(y_test,ar_forecast, "AUTOREG")
print_metrics(y_test, y_pred, "LIGHTGB Forest")

In [ ]:

# --- Feature Engineering ---

# Filter for one site
site_df = df[df['site_id'] == 'SITE_001'].copy().sort_values('date')
site_df.set_index('date', inplace=True)

# Lag features
site_df["lag_1"] = site_df['consumed_tonnes'].shift(1)
site_df["lag_3"] = site_df['consumed_tonnes'].shift(3)
site_df["lag_7"] = site_df['consumed_tonnes'].shift(7)

# Rolling statistics
site_df['rolling_mean_3'] = site_df['consumed_tonnes'].rolling(3).mean()
site_df['rolling_std_7'] = site_df['consumed_tonnes'].rolling(7).std()

# Date-based features
site_df['day_of_week'] = site_df.index.dayofweek
site_df['week_of_year'] = site_df.index.isocalendar().week.astype(int)

# Interaction features
site_df['rain_x_pour'] = site_df['rain_mm'] * site_df['planned_pour_tonnes']
site_df['temp_x_pour'] = site_df['avg_temp_c'] * site_df['planned_pour_tonnes']

# Inventory features
site_df['inventory_gap'] = (
    site_df['opening_inventory_tonnes'] + site_df['deliveries_tonnes'] - site_df['planned_pour_tonnes']
)
site_df['inventory_ratio'] = site_df['closing_inventory_tonnes'] / site_df['silo_capacity']

# Encoding categorical variables
site_df['behavior_encoded'] = site_df['behavior'].astype('category').cat.codes
site_df['cement_type_encoded'] = site_df['cement_type'].astype('category').cat.codes

# Drop NaNs from lag/rolling features
site_df.dropna(inplace=True)

# --- Define features and target ---
features = [
    "planned_pour_tonnes", "rain_mm", "avg_temp_c",
    "lag_1", "lag_3", "lag_7",
    "rolling_mean_3", "rolling_std_7",
    "rain_x_pour", "temp_x_pour",
    "inventory_gap", "inventory_ratio",
    "behavior_encoded", "cement_type_encoded",  # fixed duplicate
    "day_of_week", "week_of_year"
]

X = site_df[features]
y = site_df["consumed_tonnes"]

# --- Train-test split (time series) ---
split_index = int(len(site_df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# --- Train model ---
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
lgb =model.fit(X_train, y_train)

# Predict
y_pred = lgb.predict(X_test)

# --- Evaluate ---
def print_metrics(y_true, y_pred, label):
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{label} - MAPE: {mape:.2f}%, RMSE: {rmse:.2f} tonnes")

print_metrics(y_test, y_pred, "LightGBM (Enhanced Features)")


In [ ]:
#visualisation Random Forest Forecast Vs actual
import plotly.graph_objects as go


fig = go.Figure()

# Actual Values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    mode='lines',
    name="Actual",
    line=dict(color='black')
))

# Forecast Values
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred,
    mode='lines',
    name='Forecast',
    line=dict(color='yellow',dash='dash')
))

# Layout
fig.update_layout(
    title=f'LIGHTGB FORECAST VS ACTUAL - {site_id}',
    xaxis_title="Date",
    yaxis_title="Cement Consumed (Tonnes)",
    legend=dict(x=0, y=1),
    template='plotly_white',
    width=1000,
    height=450
)

fig.show()

In [ ]:
# import feature in the dataset

# feat_important = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
# print(feat_important.head(10))

# For LightGBM
feat_important = pd.Series(
    model.feature_importances_, 
    index=model.feature_name_
).sort_values(ascending=False)

print(feat_important.head(10))

In [ ]:
import matplotlib.pyplot as plt

# Get feature importances
feat_important = pd.Series(
    model.feature_importances_,
    index=model.feature_name_
).sort_values(ascending=False)

# Print top features
print("Top 10 features:\n", feat_important.head(10))



In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10,6))
feat_important.head(10).plot(
    kind='bar', color='skyblue', edgecolor='black', ax=ax
)
ax.set_title("Top 10 Feature Importances - LightGBM")
ax.set_ylabel("Importance Score")
ax.set_xlabel("Features")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# feature important visualisation

# Feature importance
importance = pd.Series(lgb.feature_importances_, index=features).sort_values()

# Plot
importance.plot(kind='barh', figsize=(12,5), title='Feature Importance - Random Forest')
plt.tight_layout()
plt.show()

In [ ]:
lgb.score(X_train,y_train)

In [ ]:

# Parameters
silo_capacity = 1000
initial_inventory = 600
lead_time_days = 3
reorder_threshold = 0.2 * silo_capacity
target_inventory = 0.8 * silo_capacity
buffer_rain_threshold = 10   # mm rainfall threshold
buffer_increase = 0.1

# DataFrame to store result
df_sim = pd.DataFrame({
    "date": X_test.index,
    "forecasted_consumption": y_pred,
    "rain_forecast_mm": X_test["rain_mm"].values
})



In [ ]:
df_sim.isnull().sum()

In [ ]:
# df_sim["sim_inventory"] = np.nan
# df_sim['reoder_flag'] = False
# df_sim['recommended_delivery_quantity'] = 0.0
# df_sim['buffer_applied'] = False




# Initialize columns
df_sim["sim_inventory"] = 0.0
df_sim["reorder_flag"] = False
df_sim["buffer_applied"] = False
df_sim["recommended_delivery_date"] = pd.NaT
df_sim["recommended_delivery_quantity"] = 0.0



In [ ]:


# Initialize inventory
inventory = initial_inventory
delivery_queue = {}

# Simulation loop
for i, row in df_sim.iterrows():
    today = row["date"]

    # If a delivery is due, add it to inventory
    if today in delivery_queue:
        inventory += delivery_queue[today]
        inventory = min(inventory, silo_capacity)
        del delivery_queue[today]

    # Subtract today's consumption
    consumption = row["forecasted_consumption"]
    inventory -= consumption

    # Record today's inventory
    df_sim.loc[i, "sim_inventory"] = inventory

    # Check reorder condition
    if inventory < reorder_threshold:
        df_sim.loc[i, "reorder_flag"] = True
        delivery_date = today + pd.Timedelta(days=lead_time_days)

        # Calculate delivery qty
        delivery_qty = target_inventory - inventory
        if row["rain_forecast_mm"] > buffer_rain_threshold:
            delivery_qty *= (1 + buffer_increase)
            df_sim.loc[i, "buffer_applied"] = True

        # Cap by silo capacity
        delivery_qty = min(delivery_qty, silo_capacity - inventory)
        delivery_queue[delivery_date] = delivery_qty

        # Save delivery details
        df_sim.loc[i, "recommended_delivery_date"] = delivery_date.strftime("%Y-%m-%d")
        df_sim.loc[i, "recommended_delivery_quantity"] = round(delivery_qty, 2)


In [ ]:
# KPIs
stockouts = (df_sim["sim_inventory"] < 0).sum()
service_level = 100 * (1 - stockouts / len(df_sim))
avg_inventory = df_sim["sim_inventory"].mean()
num_deliveries = df_sim["reorder_flag"].sum()

print(f"Service level: {service_level:.2f}%")
print(f"Stockouts: {stockouts}")
print(f"Average inventory: {avg_inventory:.2f} tonnes")
print(f"Number of deliveries: {num_deliveries}")

# Output
output_df = df_sim[
    ["date", "forecasted_consumption", "sim_inventory", "reorder_flag",
     "recommended_delivery_date", "recommended_delivery_quantity", "buffer_applied"]
]

output_df.head(50)

                      

In [ ]:
# import numpy as np
# import pandas as pd
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_squared_error


# ---------------- Feature Engineering ----------------
def engineer_features(df, site_id):
    # Ensure datetime
    df['date'] = pd.to_datetime(df['date'])

    # Filter by site
    site_df = df[df['site_id'] == site_id].copy().sort_values('date')
    site_df.set_index('date', inplace=True)

    # Lag features
    site_df["lag_1"] = site_df['consumed_tonnes'].shift(1)
    site_df["lag_3"] = site_df['consumed_tonnes'].shift(3)
    site_df["lag_7"] = site_df['consumed_tonnes'].shift(7)
    
    # Rolling statistics
    site_df['rolling_mean_3'] = site_df['consumed_tonnes'].rolling(3).mean()
    site_df['rolling_std_7'] = site_df['consumed_tonnes'].rolling(7).std()

    # Date-based features
    site_df['day_of_week'] = site_df.index.dayofweek
    site_df['week_of_year'] = site_df.index.isocalendar().week.astype(int)

    # Interaction features
    site_df['rain_x_pour'] = site_df['rain_mm'] * site_df['planned_pour_tonnes']
    site_df['temp_x_pour'] = site_df['avg_temp_c'] * site_df['planned_pour_tonnes']

    # Inventory features
    site_df['inventory_gap'] = (
        site_df['opening_inventory_tonnes'] + site_df['deliveries_tonnes'] - site_df['planned_pour_tonnes']
    )
    site_df['inventory_ratio'] = site_df['closing_inventory_tonnes'] / site_df['silo_capacity']

    # Encoding categorical variables
    site_df['behavior_encoded'] = site_df['behavior'].astype('category').cat.codes
    site_df['cement_type_encoded'] = site_df['cement_type'].astype('category').cat.codes

    # Drop NaNs from lag/rolling features
    site_df.dropna(inplace=True)

    return site_df


# ---------------- Model Training ----------------
def train_rf_forecast(site_df):
    # Features
    features = [
        "planned_pour_tonnes", "rain_mm", "avg_temp_c",
        "lag_1", "lag_3", "lag_7",
        "rolling_mean_3", "rolling_std_7",
        "rain_x_pour", "temp_x_pour",
        "inventory_gap", "inventory_ratio",
        "behavior_encoded", "cement_type_encoded",
        "day_of_week", "week_of_year"
    ]

    X = site_df[features]
    y = site_df["consumed_tonnes"]

    # Train-test split (time series)
    # split_index = int(len(X) * 0.8)
    # X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    # y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    X_train, X_test,y_train, y_test=train_test_split(X,y, test_size =0.2,random_state = 42)

    # Train model
    model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    return model, X_test, y_test, y_pred


model, X_test, y_test, y_pred = train_rf_forecast(site_df)
print_metrics(y_test, y_pred, "LightGBM (Enhanced Features)")





In [ ]:
def simulate_inventory(
    df_sim,
    initial_inventory,
    silo_capacity,
    reorder_threshold,
    target_inventory,
    lead_time_days=2,
    buffer_rain_threshold=10,
    buffer_increase=0.1
):


    # Add new columns
    df_sim = df_sim.copy()
    df_sim["sim_inventory"] = np.nan
    df_sim["reorder_flag"] = False
    df_sim["buffer_applied"] = False
    df_sim["recommended_delivery_date"] = None
    df_sim["recommended_delivery_quantity"] = 0.0

    # Simulation loop
    for i, row in df_sim.iterrows():
        today = row["date"]

        # If a delivery is due today, add it
        if today in delivery_queue:
            inventory += delivery_queue[today]
            inventory = min(inventory, silo_capacity)  # cap
            del delivery_queue[today]

        # Subtract today's consumption
        consumption = row["forecasted_consumption"]
        inventory -= consumption

        # Record today's inventory
        df_sim.at[i, "sim_inventory"] = inventory

        # Check reorder condition
        if inventory < reorder_threshold:
            df_sim.at[i, "reorder_flag"] = True
            delivery_date = today + pd.Timedelta(days=lead_time_days)

            # Calculate delivery qty
            delivery_qty = target_inventory - inventory
            if row["rain_forecast_mm"] > buffer_rain_threshold:
                delivery_qty *= (1 + buffer_increase)
                df_sim.at[i, "buffer_applied"] = True

            # Cap delivery by silo capacity
            delivery_qty = min(delivery_qty, silo_capacity - inventory)

            # Schedule delivery
            delivery_queue[delivery_date] = delivery_qty

            # Save delivery details
            df_sim.at[i, "recommended_delivery_date"] = delivery_date.strftime("%Y-%m-%d")
            df_sim.at[i, "recommended_delivery_quantity"] = round(delivery_qty, 2)

    return df_sim


In [ ]:


def run_pipeline(df, site_metadata, lead_time_days=2, metadata_file="pipeline_metadata.json"):
    all_metadata = {}

    for site_id, config in site_metadata.items():
        # 1. Feature engineering
        site_df = engineer_features(df, site_id)
        if site_df.empty:
            continue

        # 2. Train & forecast
        model, results = train_rf_forecast(site_df)

        # 3. Metrics
        metrics = compute_metrics(results["actual_consumption"], results["forecasted_consumption"])

        # 4. Simulate inventory
        sim_results = simulate_inventory(
            results,
            initial_inventory=config["initial_inventory"],
            silo_capacity=config["silo_capacity"],
            reorder_threshold=config["reorder_threshold"],
            target_inventory=config["target_inventory"],
            lead_time_days=lead_time_days
        )

        # 5. Collect metadata
        all_metadata[site_id] = {
            "timestamp": str(datetime.datetime.now()),
            "model_params": model.get_params(),
            "metrics": metrics,
            "inventory_settings": config
        }

    # Save all site metadata
    with open(metadata_file, "w") as f:
        json.dump(all_metadata, f, indent=4)

    print(f"✅ Metadata saved for {len(all_metadata)} sites to {metadata_file}")
    return all_metadata



In [ ]:
df['site_id'].unique()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from lightgbm import LGBMRegressor

# Calculate MAPE for each site in the pipeline
performance_summary = []

for site_id in df['site_id'].unique():
    print(f"Evaluating model for {site_id}...")
    
    # Prepare site data
    site_df = engineer_features(df, site_id)
    
    if len(site_df) < 50:
        print(f"Skipping {site_id}, not enough data")
        continue
        
    # Features and target
    features = [
        'planned_pour_tonnes', 'rain_mm', 'avg_temp_c',
        'lag_1', 'lag_3', 'lag_7',
        'rolling_mean_3', 'rolling_std_7',
        'rain_x_pour', 'temp_x_pour',
        'inventory_gap', 'inventory_ratio',
        'behavior_encoded', 'cement_type_encoded'
    ]
    X = site_df[features]
    y = site_df['consumed_tonnes']

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train model
    model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Calculate metrics - handling zero values properly
    mask = y_test != 0  # avoid division by zero
    if mask.sum() > 0:  
        mape = mean_absolute_percentage_error(y_test[mask], y_pred[mask]) * 100
    else:
        mape = 0  
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Store results
    performance_summary.append({
        'site_id': site_id,
        'test_samples': len(y_test),
        'non_zero_samples': mask.sum(),
        'mape': mape,
        'rmse': rmse,
        'avg_consumption': y_test.mean()
    })

# Create performance dataframe
performance_df = pd.DataFrame(performance_summary)

# Display overall performance
print("\n" + "="*60)
print("OVERALL MODEL PERFORMANCE SUMMARY")
print("="*60)
print(f"Average MAPE across all sites: {performance_df['mape'].mean():.2f}%")
print(f"Median MAPE across all sites: {performance_df['mape'].median():.2f}%")
print(f"Sites achieving MAPE ≤ 15%: {(performance_df['mape'] <= 15).sum()}/{len(performance_df)}")
print(f"Sites with MAPE > 15%: {(performance_df['mape'] > 15).sum()}/{len(performance_df)}")

# Display detailed performance table
print("\nDetailed Performance by Site:")
print(performance_df.sort_values('mape').to_string(index=False))

# Identify sites that need attention
problem_sites = performance_df[performance_df['mape'] > 15]
if not problem_sites.empty:
    print("\n" + "!"*60)
    print("SITES NEEDING ATTENTION (MAPE > 15%):")
    print("!"*60)
    print(problem_sites[['site_id', 'mape', 'avg_consumption', 'test_samples']].to_string(index=False))
else:
    print("\n🎉 ALL SITES ACHIEVED TARGET MAPE OF 15% OR BETTER! 🎉")


In [ ]:
# ---------------- Metadata Generator ----------------
def generate_site_metadata():
    site_metadata = {}

    site_configs = [
        (1200, 674, 240, 960), (2500, 1421, 500, 2000), (1000, 592, 200, 800),
        (1500, 899, 300, 1200), (1200, 640, 240, 960), (1800, 1070, 360, 1440),
        (2000, 1198, 400, 1600), (1000, 612, 200, 800), (1500, 891, 300, 1200),
        (2200, 1387, 440, 1760), (1200, 701, 240, 960), (1000, 603, 200, 800),
        (1800, 1084, 360, 1440), (1500, 902, 300, 1200), (2000, 1235, 400, 1600),
        (1200, 689, 240, 960), (2500, 1453, 500, 2000), (1500, 874, 300, 1200),
        (1800, 1102, 360, 1440), (1000, 583, 200, 800), (2200, 1364, 440, 1760),
        (1200, 651, 240, 960), (1500, 900, 300, 1200), (1800, 1111, 360, 1440),
        (2000, 1180, 400, 1600), (1000, 594, 200, 800), (1200, 662, 240, 960),
        (1500, 917, 300, 1200), (1800, 1095, 360, 1440), (2000, 1210, 400, 1600)
    ]

    for i, (capacity, inventory, threshold, target) in enumerate(site_configs, start=1):
        site_id = f"SITE_{i:03d}"
        site_metadata[site_id] = {
            "silo_capacity": capacity,
            "initial_inventory": inventory,
            "reorder_threshold": threshold,
            "target_inventory": target
        }

    return site_metadata


# ---------------- Main ----------------
if __name__ == "__main__":
    metadata = generate_site_metadata()
    for site, data in metadata.items():
        print(f"{site}: {data}")

 # Convert dict → DataFrame
    metadata_df = pd.DataFrame.from_dict(metadata, orient="index").reset_index()
    metadata_df.rename(columns={"index": "site_id"}, inplace=True)

   
    # Save to parquet
    metadata_df.to_parquet("site_metadata.parquet", index=False)

    print(" Metadata saved to site_metadata.parquet")

       
        



In [ ]:
performance_df = pd.DataFrame(performance_summary)

In [ ]:
performance_df.head(30)